<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=913a2fe3218c27e54e021d8414a943af3ed4c0b25caa19263466e2a46b028750
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-21 13:48:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.35 C
-------------------
Today PnL: -73.73 K (-0.54%)
Current PnL: -33.79 L (-21.55%)
CY Booked + Current PnL: -19.50 L (-12.44%)
-------------------
Total profit:  1.19 L
Total loss:  -34.98 L
-------------------
Total Booked + Current PnL: 7.48 L (5.76%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.56%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 99.15 L (73.29%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.98,13.11,7.33,21.40,22006.0,34796.0,300222.0,4.93,47.0,X-LC,7.10,14.0,1.58,2.22,20.27,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-1.69,-24.61,32.77,0.09,24896.0,-24804.0,75973.0,76.31,33.0,M-SC,5.04,253.0,-1.00,0.56,0.00,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
49,MASFIN,398.61,-0.81,-7.78,32.34,22.05,29222.0,-7620.0,90360.0,-20.51,39.0,H-SC,4.70,168.0,-0.26,0.67,31.34,XR,ATH,FINANCE
32,HCLTECH,1908.19,-0.55,8.90,13.72,23.83,36145.0,21522.0,263446.0,13.61,57.0,X-LC,6.33,13.0,0.60,1.95,26.40,X40,ATH,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,4.27,-4.03,115.41,106.72,136599.0,-4976.0,118360.0,-52.54,52.0,H-SC,8.67,138.0,-0.04,0.88,18.40,AR,ATH,MISC
4,ALKYLAMINE,4546.37,2.88,-32.23,185.72,93.63,127073.0,-32541.0,68422.0,-27.39,48.0,H-SC,16.52,143.0,-0.26,0.51,3.44,SR,ATH,CHEMICALS
26,GILLETTE,11206.78,2.63,-4.29,38.35,32.42,93196.0,-10881.0,243015.0,-21.27,49.0,X-SC,12.22,87.0,-0.12,1.80,10.17,X40,ATH,FMCG
20,DABUR,735.00,2.02,0.75,42.64,43.70,106566.0,1852.0,249920.0,-7.59,55.0,X-MC,2.36,73.0,0.02,1.85,15.91,XY24,BTT,FMCG
0,5PAISA,593.00,1.44,-30.86,63.29,12.90,95612.0,-67426.0,151070.0,122.36,53.0,H-SC,7.40,173.0,-0.71,1.12,25.61,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1856.0,-5.30,3.60,40.15,45.20,79756.0,6903.0,198645.0,-6.47,33.0,X-MC,5.67,80.0,0.09,1.47,10.64,XY25,NTT,AC
40,INDUSINDBK,1800.0,-4.70,-33.17,98.86,32.89,51006.0,-25612.0,51594.0,-736.93,54.0,L-MC,10.21,259.0,-0.50,0.38,42.08,XR,NTT,BANKS
44,JCHAC,2282.0,-4.21,-41.01,69.49,-0.01,52394.0,-52407.0,75398.0,15334.78,29.0,M-SC,14.06,235.0,-1.00,0.56,0.00,OX40N,BTT,AC
56,RAJOOENG,143.1,-3.83,-34.65,113.65,39.61,76123.0,-35520.0,66980.0,-55.84,48.0,H-SC,28.75,136.0,-0.47,0.50,8.42,AR,ATH,MISC
24,EASEMYTRIP,26.4,-2.99,-64.18,306.78,45.70,194281.0,-113486.0,63329.0,-34.35,17.0,M-SC,20.30,207.0,-0.58,0.47,0.00,XY24,NTT,TRAVEL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.20,-10.04,131.11,107.91,174148.0,-14818.0,132826.0,-26.63,32.0,M-SC,10.03,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.46,-21.04,26.66,0.02,36748.0,-36721.0,137838.0,110.38,37.0,M-SC,8.78,240.0,-1.00,1.02,19.90,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-1.23,-23.32,72.18,32.03,72967.0,-30745.0,101090.0,-17.04,45.0,H-SC,6.44,157.0,-0.42,0.75,13.23,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-1.69,-24.61,32.77,0.09,24896.0,-24804.0,75973.0,76.31,33.0,M-SC,5.04,253.0,-1.00,0.56,0.00,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.82,-25.67,52.66,13.47,105556.0,-69219.0,200448.0,-69.48,37.0,H-SC,4.40,159.0,-0.66,1.48,1.92,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.58,-2.18,76.10,72.27,126142.0,-3690.0,165758.0,-16.06,31.0,M-LC,4.50,101.0,-0.03,1.23,3.86,XR,NTT,IT
38,INDIAMART,4810.62,4.27,-4.03,115.41,106.72,136599.0,-4976.0,118360.0,-52.54,52.0,H-SC,8.67,138.0,-0.04,0.88,18.40,AR,ATH,MISC
25,FINCABLES,1641.55,-2.20,-10.04,131.11,107.91,174148.0,-14818.0,132826.0,-26.63,32.0,M-SC,10.03,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.20,-7.71,46.97,35.63,90844.0,-16166.0,193408.0,-20.73,35.0,H-MC,4.73,119.0,-0.18,1.43,7.67,AR,ATH,PHARMA
37,IEX,219.00,-1.09,-12.54,69.94,48.63,122309.0,-25077.0,174877.0,-41.93,37.0,H-SC,13.56,137.0,-0.21,1.29,0.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.88,-32.52,132.95,57.19,292579.0,-106060.0,220067.0,-68.10,15.0,X-MC,12.94,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
43,ITC,452.00,-0.05,-20.06,38.59,10.78,74132.0,-48216.0,192102.0,-52.19,16.0,X-LC,12.78,1.0,-0.65,1.42,0.0,X40,NTT,FMCG
58,RELAXO,1176.00,-1.01,-53.23,225.63,52.31,153194.0,-77264.0,67896.0,-50.25,17.0,X-SC,9.18,92.0,-0.50,0.50,0.0,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.15,-43.36,138.71,35.20,101089.0,-55792.0,72878.0,-4.40,19.0,X-SC,18.20,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.43,-10.62,50.13,34.19,95672.0,-22672.0,190848.0,-37.21,21.0,X-MC,5.45,60.0,-0.24,1.41,0.0,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,0.33,-14.42,38.52,18.55,124689.0,-54537.0,323700.0,-26.21,44.0,X-LC,0.97,3.0,-0.44,2.39,10.13,X40,ATH,IT
35,ICICIGI,2252.93,-1.70,-3.06,23.11,19.34,44829.0,-6129.0,193980.0,-23.57,31.0,X-MC,2.16,71.0,-0.14,1.43,12.39,X40,ATH,INSURANCE
33,HINDUNILVR,2922.00,-0.13,-7.58,22.98,13.67,54598.0,-19473.0,237590.0,-14.44,52.0,X-LC,2.29,9.0,-0.36,1.76,12.16,XY25,NTT,FMCG
20,DABUR,735.00,2.02,0.75,42.64,43.70,106566.0,1852.0,249920.0,-7.59,55.0,X-MC,2.36,73.0,0.02,1.85,15.91,XY24,BTT,FMCG
45,JIOFIN,387.00,-0.30,-14.86,46.20,24.48,105171.0,-39723.0,227642.0,-16.28,25.0,X-LC,2.47,37.0,-0.38,1.68,31.95,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.88,-32.52,132.95,57.19,292579.0,-106060.0,220067.0,-68.10,15.0,X-MC,12.94,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.49,-23.47,103.16,55.47,192068.0,-57101.0,186185.0,-32.12,23.0,X-MC,15.98,63.0,-0.30,1.38,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.15,-43.36,138.71,35.20,101089.0,-55792.0,72878.0,-4.40,19.0,X-SC,18.20,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.63,-26.40,78.35,31.26,133070.0,-60928.0,169840.0,-65.62,26.0,X-MC,23.98,56.0,-0.46,1.26,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-1.66,-15.23,56.28,32.48,156481.0,-49951.0,278040.0,-19.44,29.0,X-MC,5.27,69.0,-0.32,2.06,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.20,-33.16,113.72,42.84,49408.0,-21559.0,43447.0,-56.47,33.0,X-SC,23.01,83.0,-0.44,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-1.01,-53.23,225.63,52.31,153194.0,-77264.0,67896.0,-50.25,17.0,X-SC,9.18,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.15,-43.36,138.71,35.20,101089.0,-55792.0,72878.0,-4.40,19.0,X-SC,18.20,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.02,-11.99,40.99,24.08,45796.0,-15226.0,111724.0,-19.13,28.0,X-SC,6.91,91.0,-0.33,0.83,3.60,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.06,-17.91,74.59,43.32,99727.0,-29172.0,133700.0,-29.42,45.0,X-SC,8.19,90.0,-0.29,0.99,2.66,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.05,-20.06,38.59,10.78,74132.0,-48216.0,192102.0,-52.19,16.0,X-LC,12.78,1.0,-0.65,1.42,0.00,X40,NTT,FMCG
75,TMPV,600.00,0.74,-30.24,76.29,22.98,120219.0,-68316.0,157582.0,-27.96,34.0,X-LC,3.61,2.0,-0.57,1.16,0.74,XY24,NTT,AUTO
73,TCS,4311.59,0.33,-14.42,38.52,18.55,124689.0,-54537.0,323700.0,-26.21,44.0,X-LC,0.97,3.0,-0.44,2.39,10.13,X40,ATH,IT
81,VBL,671.64,-2.18,-3.35,40.38,35.67,123260.0,-10591.0,305251.0,-14.83,42.0,X-LC,2.89,5.0,-0.09,2.26,10.04,X40N,ATH,FMCG
41,INFY,1972.00,-0.41,12.47,19.36,34.24,68127.0,39009.0,351897.0,-12.42,56.0,X-LC,7.27,6.0,0.57,2.60,21.78,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-4.70,-33.17,98.86,32.89,51006.0,-25612.0,51594.0,-736.93,54.0,L-MC,10.21,259.0,-0.50,0.38,42.08,XR,NTT,BANKS
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
5,ASIANPAINT,3460.25,0.16,-6.33,29.11,20.95,68653.0,-15928.0,235840.0,-7.24,37.0,X-LC,12.62,36.0,-0.23,1.74,27.06,X40,ATH,PAINTS
32,HCLTECH,1908.19,-0.55,8.90,13.72,23.83,36145.0,21522.0,263446.0,13.61,57.0,X-LC,6.33,13.0,0.60,1.95,26.40,X40,ATH,IT
51,NESTLEIND,1377.00,-0.98,13.11,7.33,21.40,22006.0,34796.0,300222.0,4.93,47.0,X-LC,7.10,14.0,1.58,2.22,20.27,XY25,NTT,FMCG


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.44,-30.86,63.29,12.90,95612.0,-67426.0,151070.0,122.36,53.0,H-SC,7.40,173.0,-0.71,1.12,25.61,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
40,INDUSINDBK,1800.00,-4.70,-33.17,98.86,32.89,51006.0,-25612.0,51594.0,-736.93,54.0,L-MC,10.21,259.0,-0.50,0.38,42.08,XR,NTT,BANKS
20,DABUR,735.00,2.02,0.75,42.64,43.70,106566.0,1852.0,249920.0,-7.59,55.0,X-MC,2.36,73.0,0.02,1.85,15.91,XY24,BTT,FMCG
56,RAJOOENG,143.10,-3.83,-34.65,113.65,39.61,76123.0,-35520.0,66980.0,-55.84,48.0,H-SC,28.75,136.0,-0.47,0.50,8.42,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.65
1,25,44.89
2,50,76.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.41
MC    30.73
LC    26.91
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.21
X40      24.25
X40N     12.90
XY25     10.93
XR        8.75
AR        8.48
OX40N     7.57
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.34
X-LC    23.48
H-SC    22.68
M-SC    11.00
X-SC     7.95
H-MC     4.57
M-MC     1.44
M-LC     1.23
L-LC     1.14
H-LC     1.06
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.54,-11.07,47.52
IT,12.86,-22.93,84.35
FINANCE,11.04,-19.40,69.52
MISC,7.06,-36.26,91.99
PAINTS,5.78,-17.91,35.43
ELECTRICAL,5.78,-20.31,64.38
INSURANCE,4.72,-5.98,42.24
PHARMA,3.99,-9.02,43.35
BANKS,2.78,-41.99,128.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3425232.0,21
AR,1427373.0,10
XR,1309836.0,12
X40,1230087.0,15
X40N,999561.0,9
OX40N,768335.0,10
XY25,463555.0,7
SR,290797.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3728245.0,24
X-MC,1742945.0,16
M-SC,1572403.0,15
X-LC,1035343.0,13
X-SC,845779.0,8
H-MC,461034.0,3
L-SC,185584.0,2
M-LC,126142.0,1
H-LC,90455.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1311017.0      6
           AR         952359.0      5
M-SC       XY24       892917.0      6
H-SC       XR         848286.0      7
X-MC       X40        598744.0      7
           XY24       465609.0      3
           X40N       437426.0      4
X-LC       X40        437058.0      6
M-SC       OX40N      347649.0      5
X-SC       X40N       335789.0      3
H-SC       OX40N      325786.0      4
X-SC       XY24       315705.0      3
H-SC       SR         290797.0      2
H-MC       AR         246440.0      2
X-MC       XY25       241166.0      2
X-LC       X40N       226346.0      2
           XY24       225390.0      2
H-MC       XY24       214594.0      1
X-SC       X40        194285.0      2
M-SC       XR         193718.0      2
X-LC       XY25       146549.0      3
M-SC       AR         138119.0      2
M-LC       XR         126142.0      1
L-SC       OX40N       94900.0      1
           XR          90684.0      1
H-LC       AR          90455.0      1
L-MC       XR          51006.0      1
M-MC       XY25        49998.0      1
L-LC       XY25        25842.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
